In [1]:
import numpy as np

In [2]:
arr=np.arange(0, 4*2*4)
len(arr)
arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [3]:
v=arr.reshape([4,2,4])
v

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7]],

       [[ 8,  9, 10, 11],
        [12, 13, 14, 15]],

       [[16, 17, 18, 19],
        [20, 21, 22, 23]],

       [[24, 25, 26, 27],
        [28, 29, 30, 31]]])

In [4]:
v.ndim
v.sum() #모든 요소 합

496

In [5]:
v.sum(axis=0) #행
v.shape

(4, 2, 4)

In [6]:
v.sum(axis=1)#열

array([[ 4,  6,  8, 10],
       [20, 22, 24, 26],
       [36, 38, 40, 42],
       [52, 54, 56, 58]])

In [7]:
v.sum(axis=2)#깊이

array([[  6,  22],
       [ 38,  54],
       [ 70,  86],
       [102, 118]])

In [8]:
#tf.nn.dynamic_rnn  tf.transpose(대상, [0,2,1])

In [9]:
myTensor=np.arange(2*3*4).reshape(2,3,4)
myTensor
#yTensor.shape #(2,3,4)

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [10]:
np.transpose(myTensor).shape #(4, 3, 2)

(4, 3, 2)

In [11]:
np.transpose(myTensor, [1,2,0]).shape #234 ->342

(3, 4, 2)

In [12]:
import tensorflow as tf

In [13]:
from tensorflow.examples.tutorials.mnist import input_data

In [18]:
mnist=input_data.read_data_sets("./mnist2/data/", one_hot=True)

W0821 21:13:36.287910  1796 deprecation.py:323] From <ipython-input-18-56674d307c35>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0821 21:13:36.287910  1796 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0821 21:13:36.289904  1796 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

Extracting ./mnist2/data/train-images-idx3-ubyte.gz


W0821 21:13:36.539237  1796 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0821 21:13:36.541233  1796 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0821 21:13:36.591099  1796 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version

Extracting ./mnist2/data/train-labels-idx1-ubyte.gz
Extracting ./mnist2/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist2/data/t10k-labels-idx1-ubyte.gz


In [14]:
learning_rate=0.001
total_epoch=30
batch_size=128

In [15]:
n_input=28 #가로 픽셀수
n_step=28 #세로 픽셀수(입력 단계, step)
n_hidden=128 
n_class=10

In [16]:
tf.reset_default_graph()
x=tf.placeholder(tf.float32, [None, n_step, n_input])
y=tf.placeholder(tf.float32, [None,n_class])
w=tf.Variable(tf.random_normal([n_hidden, n_class]))
b=tf.Variable(tf.random_normal([n_class]))

In [20]:
cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states=tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
#Y:[batch_size, nclass]
#                0     1         2
#outputs:[batch_size, n_step, n_hidden]
# => [n_step, batch_size, n_hidden]
#=> [batch_size, n_hidden]
outputs=tf.transpose(outputs,[1,0,2]) #[n_step, batch_size, n_hidden]
outputs=outputs[-1] # [batch_size, n_hidden]
model=tf.matmul(outputs,w)+b
#tf.matmul([batch_size, n_hidden],[n_hidden,n_class])
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=model, labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

total_batch=int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
    total_cost=0
    for i in range(total_batch):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape((batch_size, n_step, n_input))
        # [batch_size, nstep, ninput]        
        _, cv=sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys})
        total_cost+=cv
    print("에폭:", "%4d" % (epoch+1),
         'avg cost=', "{:.3f}".format(total_cost/total_batch))


ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [59]:
is_correct=tf.equal(tf.argmax(model,1), tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size=len(mnist.test.images)

test_xs=mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys=mnist.test.labels

print("정확도:", sess.run(accuracy, feed_dict={x:test_xs, y:test_ys}))
    

정확도: 0.9676


In [58]:
# {'a':0, ..., 'z':25} 
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']